In [1]:
import cv2
import copy
import pytesseract
import tempfile
from pytesseract import Output

* see parameters of pytesseract like for hindi language
* accuracy

# Input Image

In [2]:
img_path = 'images.jpeg'

In [3]:
# read image
im = cv2.imread(img_path)
im = cv2.resize(im,(800,900))
org = cv2.imread(img_path)
org = cv2.resize(im,(800,900))

# Image Preprocessing

In [4]:
# gray scale
def gray(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(r"./preprocess/img_gray.png",img)
    return img

# blur
def blur(img) :
    img_blur = cv2.GaussianBlur(img,(5,5),0)
    cv2.imwrite(r"./preprocess/img_blur.png",img)    
    return img_blur

# threshold
def threshold(img):
    #pixels with value below 100 are turned black (0) and those with higher value are turned white (255)
    img = cv2.threshold(img, 100, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)[1]    
    cv2.imwrite(r"./preprocess/img_threshold.png",img)
    return img

In [5]:
def show_image(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()         

In [6]:
im = gray(im)
im = blur(im)
im = threshold(im)
# contours, _ = cv2.findContours(im_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

In [7]:
# show_image(im)

# Merge Bounding Boxes

In [8]:
#Distance definition  between text to be merge
dist_limit = 50

#Generate two text boxes a larger one that covers them
def merge_boxes(box1, box2):
    return [min(box1[0], box2[0]), 
         min(box1[1], box2[1]), 
         max(box1[2], box2[2]),
         max(box1[3], box2[3])]

def check(point_x,point_y,mat):
    mat_ymin, mat_xmin, mat_ymax, mat_xmax = mat
    if(point_x >= mat_xmin and point_x <= mat_xmax):
        if(point_y >= mat_ymin and point_y <= mat_ymax):
            return True
    return False

def check_overlap(mat1,mat2):
    mat1_ymin, mat1_xmin, mat1_ymax, mat1_xmax = mat1
    mat2_ymin, mat2_xmin, mat2_ymax, mat2_xmax = mat2
    flag1 = check(mat1_xmin,mat1_ymin,mat2) or check(mat1_xmax,mat1_ymax,mat2) or check(mat1_xmin,mat1_ymax,mat2) or check(mat1_xmax,mat1_ymin,mat2)
    flag2 = check(mat2_xmin,mat2_ymin,mat1) or check(mat2_xmax,mat2_ymax,mat1) or check(mat2_xmin,mat2_ymax,mat1) or check(mat2_xmax,mat2_ymin,mat1)
    return flag1 or flag2

#Computer a Matrix similarity of distances of the text and object
def calc_sim(text, obj):
    # text: ymin, xmin, ymax, xmax
    # obj: ymin, xmin, ymax, xmax
    text_ymin, text_xmin, text_ymax, text_xmax = text
    obj_ymin, obj_xmin, obj_ymax, obj_xmax = obj

    x_dist = min(abs(text_xmin-obj_xmin), abs(text_xmin-obj_xmax), abs(text_xmax-obj_xmin), abs(text_xmax-obj_xmax))
    y_dist = min(abs(text_ymin-obj_ymin), abs(text_ymin-obj_ymax), abs(text_ymax-obj_ymin), abs(text_ymax-obj_ymax))

    dist = x_dist + y_dist
#     print(x_dist,y_dist)
    if(dist <= dist_limit):
        return True
    return check_overlap(text,obj)

#Principal algorithm for merge text 
def merge_algo(texts, texts_boxes):
    for i, (text_1, text_box_1) in enumerate(zip(texts, texts_boxes)):
        for j, (text_2, text_box_2) in enumerate(zip(texts, texts_boxes)):
            if j <= i:
                continue
            # Create a new box if a distances is less than disctance limit defined 
            if calc_sim(text_box_1, text_box_2):
            # Create a new box  
                new_box = merge_boxes(text_box_1, text_box_2)            
             # Create a new text string 
                new_text = text_1 + ' ' + text_2
                print(new_text)
                texts[i] = new_text
                #delete previous text 
                del texts[j]
                texts_boxes[i] = new_box
                #delete previous text boxes
                del texts_boxes[j]
                #return a new boxes and new text string that are close
                return True, texts, texts_boxes

    return False, texts, texts_boxes


def merge(texts,texts_boxes):
    #Copy of the text and object arrays
    texts_copied = copy.deepcopy(texts)
    texts_boxes_copied = copy.deepcopy(texts_boxes)
    need_to_merge = True
    #Merge full text 
    while need_to_merge:
        need_to_merge, texts_copied, texts_boxes_copied = merge_algo(texts_copied, texts_boxes_copied)
    return texts_copied,texts_boxes_copied

# OCR

In [9]:
def simple_ocr(img):
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    org_cpy = copy.deepcopy(org)
    print(d)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if(d['text'][i] == '' or d['text'][i].isspace()):
            continue
        if int(d['conf'][i]) > 0:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(org_cpy, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()         

In [10]:
def ocr(img):
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
    texts = []
    texts_boxes = []
        
    for i in range(n_boxes):
        if(d['text'][i] == '' or d['text'][i].isspace()):
            continue
        if int(d['conf'][i]) > 60:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cord = [y,x,y+h,x+w]
            texts.append(d['text'][i])
            texts_boxes.append(cord)
              
    new_texts,new_texts_boxes = merge(texts,texts_boxes)
#     print(new_texts)
#     print(new_texts_boxes)
    return new_texts,new_texts_boxes

In [11]:
def plot(text,text_boxes):
    n_boxes = len(text)
    org_cpy = copy.deepcopy(org)
    flag = 0
    for i in range(n_boxes):
        (y1, x1, y2, x2) = text_boxes[i]
        print(text[i])
        print(text_boxes[i])
        flag = 1
        img = cv2.rectangle(org_cpy, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    if flag == 1:
        cv2.imshow('img', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()         

In [ ]:
# simple_ocr(im)

In [13]:
texts,texts_boxes = ocr(im)

In [14]:
plot(texts,texts_boxes)